## Looking at ongoing trials registered in BCC, SCC, and urinary incontinence in women
### Preliminary estimates for NLM renewal


In [1]:
from trialstreamer.dbutil import db
import psycopg2
import tqdm
import json
import os

/home/iain/anaconda3/envs/rr3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [70]:
queries = ["basal cell carcinoma", "squamous cell carcinoma", "urinary incontinence"]

exclude_strings = [["diagnos", "signatures", 'non-interventional', 'database', 'retrospective'],
                   ["esophag", "head", "oral", "pharyn", 'laryng', "lung", "cervical", "mouth", "hnscc", "nsclc", "neck", "buccal", 'hpv', 'anal', 'tongue', 'glott', 'digestive', 'penis', 'penile', 'extremities', 'sino', 'sinus', 'nasal', 'cervix', 'actinic keratosis', 'organ transplant', 'prognostic'],
                   ["male", "prevalence", "peni", 'men', 'prosta', 'child']]

base_path = "/home/iain/Dropbox/documents/nlm-renewal/brown-systematic-reviews/trial-registry-searches"

In [71]:
# initial searches for manual screening of relevant studies
for q, e in zip(queries, exclude_strings):
    cur = db.cursor(cursor_factory=psycopg2.extras.DictCursor)
    cur.execute("select ti, year, ictrp_data from ictrp where lower(ti) like %s;", ("%{}%".format(q),))
    records = cur.fetchall()
    with open(os.path.join(base_path, "1-search-retrieval-{}".format(q.replace(" ", '-'))), 'w') as f:
        filt = [r['ti'].strip() for r in records if r['year'] is not None and int(r['year']) > 2012]
        filt = [r for r in filt if all((e_x not in r.lower() for e_x in e))]
        filt = list(set(filt)) # remove dupes
        f.write("{} RECORDS retrieved for {}\n\n\n".format(len(filt), q))
        f.write('\n\n'.join(["{}.\t{}".format(ix+1, fr) for ix, fr in enumerate(filt)]))
